In [9]:
from ortools.sat.python import cp_model


def solve_sudoku(field: list[list]):
    """Solves the sudoku problem with the CP-SAT solver."""
    # Create the model.
    model = cp_model.CpModel()

    dimension = len(field)
    cell_size = int(dimension**(1/2))
    line_size = cell_size**2
    line = list(range(0, line_size))
    cell = list(range(0, cell_size))

    initial_grid = field

    grid = {}
    for i in line:
        for j in line:
            grid[(i, j)] = model.new_int_var(1, line_size, "grid %i %i" % (i, j))

    # AllDifferent on rows.
    for i in line:
        model.add_all_different(grid[(i, j)] for j in line)

    # AllDifferent on columns.
    for j in line:
        model.add_all_different(grid[(i, j)] for i in line)

    # AllDifferent on cells.
    for i in cell:
        for j in cell:
            one_cell = []
            for di in cell:
                for dj in cell:
                    one_cell.append(grid[(i * cell_size + di, j * cell_size + dj)])

            model.add_all_different(one_cell)

    # Initial values.
    for i in line:
        for j in line:
            if initial_grid[i][j]:
                model.add(grid[(i, j)] == initial_grid[i][j])

    # Solves and prints out the solution.
    solver = cp_model.CpSolver()
    status = solver.solve(model)
    if status == cp_model.OPTIMAL:
        for i in line:
            print([int(solver.value(grid[(i, j)])) for j in line])


# 3x3 example
example_3x3 = [
    [0, 2, 0, 0, 7, 8, 0, 0, 6],
    [0, 0, 9, 1, 5, 0, 0, 8, 4],
    [5, 6, 8, 0, 0, 0, 7, 0, 0],
    [0, 0, 5, 0, 8, 0, 6, 0, 9],
    [0, 3, 0, 0, 0, 9, 0, 0, 8],
    [0, 0, 0, 6, 0, 0, 0, 4, 3],
    [8, 0, 3, 5, 6, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 3, 0, 0, 0],
    [2, 9, 0, 0, 4, 0, 0, 0, 0]]

example_4x4 = [
    [0, 2, 5, 3, 4, 6, 7, 8, 0, 13, 11, 10, 0, 12, 15, 16],
    [7, 6, 0, 0, 1, 0, 3, 0, 12, 14, 0, 16, 5, 8, 13, 0],
    [0, 4, 12, 0, 5, 0, 16, 15, 0, 2, 3, 0, 7, 0, 10, 0],
    [0, 0, 15, 16, 10, 0, 12, 13, 0, 5, 7, 8, 0, 0, 3, 6],
    [2, 0, 3, 4, 0, 5, 0, 7, 8, 10, 0, 11, 13, 14, 16, 15],
    [5, 7, 6, 8, 2, 1, 0, 3, 13, 0, 0, 0, 12, 0, 11, 9],
    [10, 11, 0, 0, 8, 0, 15, 0, 0, 0, 4, 0, 3, 0, 6, 7],
    [0, 9, 0, 0, 0, 0, 14, 10, 0, 7, 6, 0, 2, 0, 4, 8],
    [3, 5, 0, 0, 0, 0, 0, 11, 10, 8, 0, 0, 0, 16, 14, 0],
    [0, 8, 7, 6, 3, 9, 1, 0, 15, 16, 0, 13, 10, 11, 0, 12],
    [11, 13, 14, 0, 15, 16, 0, 5, 3, 4, 1, 7, 0, 0, 0, 0],
    [15, 16, 9, 10, 14, 0, 0, 0, 6, 0, 2, 5, 0, 3, 7, 1],
    [6, 3, 2, 1, 0, 0, 0, 14, 0, 0, 5, 4, 0, 13, 0, 0],
    [13, 0, 4, 5, 12, 3, 0, 0, 0, 0, 8, 0, 0, 7, 1, 0],
    [14, 0, 8, 0, 0, 10, 5, 4, 11, 6, 0, 1, 9, 0, 2, 0],
    [0, 0, 11, 9, 13, 0, 8, 0, 14, 3, 10, 0, 6, 4, 0, 0]]


solve_sudoku(example_4x4)

[1, 2, 5, 3, 4, 6, 7, 8, 9, 13, 11, 10, 14, 12, 15, 16]
[7, 6, 10, 11, 1, 2, 3, 9, 12, 14, 15, 16, 5, 8, 13, 4]
[8, 4, 12, 13, 5, 14, 16, 15, 1, 2, 3, 6, 7, 9, 10, 11]
[9, 14, 15, 16, 10, 11, 12, 13, 4, 5, 7, 8, 1, 2, 3, 6]
[2, 1, 3, 4, 6, 5, 9, 7, 8, 10, 12, 11, 13, 14, 16, 15]
[5, 7, 6, 8, 2, 1, 4, 3, 13, 15, 16, 14, 12, 10, 11, 9]
[10, 11, 13, 14, 8, 12, 15, 16, 2, 1, 4, 9, 3, 5, 6, 7]
[12, 9, 16, 15, 11, 13, 14, 10, 5, 7, 6, 3, 2, 1, 4, 8]
[3, 5, 1, 2, 7, 4, 6, 11, 10, 8, 9, 12, 15, 16, 14, 13]
[4, 8, 7, 6, 3, 9, 1, 2, 15, 16, 14, 13, 10, 11, 5, 12]
[11, 13, 14, 12, 15, 16, 10, 5, 3, 4, 1, 7, 8, 6, 9, 2]
[15, 16, 9, 10, 14, 8, 13, 12, 6, 11, 2, 5, 4, 3, 7, 1]
[6, 3, 2, 1, 9, 15, 11, 14, 7, 12, 5, 4, 16, 13, 8, 10]
[13, 10, 4, 5, 12, 3, 2, 6, 16, 9, 8, 15, 11, 7, 1, 14]
[14, 12, 8, 7, 16, 10, 5, 4, 11, 6, 13, 1, 9, 15, 2, 3]
[16, 15, 11, 9, 13, 7, 8, 1, 14, 3, 10, 2, 6, 4, 12, 5]
